In [ ]:
!pip install -qqq git+https://github.com/srush/MiniChain@gradio

In [ ]:
desc = """
### Chat

A chat-like example for multi-turn chat with state. [[Code](https://github.com/srush/MiniChain/blob/main/examples/chat.py)]

(Adapted from [LangChain](https://langchain.readthedocs.io/en/latest/modules/memory/examples/chatgpt_clone.html)'s version of this [blog post](https://www.engraved.blog/building-a-virtual-machine-inside/).)

"""

In [ ]:
from dataclasses import dataclass, replace
from typing import List, Tuple
from minichain import OpenAI, prompt, show

Generic stateful Memory

In [ ]:
MEMORY = 2

In [ ]:
@dataclass
class State:
    memory: List[Tuple[str, str]]
    human_input: str = ""

    def push(self, response: str) -> "State":
        memory = self.memory if len(self.memory) < MEMORY else self.memory[1:]
        return State(memory + [(self.human_input, response)])

Chat prompt with memory

In [ ]:
@prompt(OpenAI(), template_file="chat.pmpt.tpl")
def chat_prompt(model, state: State) -> State:
    out = model(state)
    result = out.split("Assistant:")[-1]
    return state.push(result)

In [ ]:
examples = [
    "ls ~",
    "cd ~",
    "{Please make a file jokes.txt inside and put some jokes inside}",
    """echo -e "x=lambda y:y*5+3;print('Result:' + str(x(6)))" > run.py && python3 run.py""",
    """echo -e "print(list(filter(lambda x: all(x%d for d in range(2,x)),range(2,3**10)))[:10])" > run.py && python3 run.py""",
    """echo -e "echo 'Hello from Docker" > entrypoint.sh && echo -e "FROM ubuntu:20.04\nCOPY entrypoint.sh entrypoint.sh\nENTRYPOINT [\"/bin/sh\",\"entrypoint.sh\"]">Dockerfile && docker build . -t my_docker_image && docker run -t my_docker_image""",
    "nvidia-smi"
]

In [ ]:
gradio = show(lambda command, state: chat_prompt(replace(state, human_input=command)),
              initial_state=State([]),
              subprompts=[chat_prompt],
              examples=examples,
              out_type="json",
              description=desc,
)
if __name__ == "__main__":
    gradio.launch()